In [ ]:
# Donwload required data
import yfinance as yf

# ticker_symbol = 'TSLA'
# tsla = yf.download('TSLA', start='2016-01-01', end='2016-12-31')
# tsla.to_csv(f'./downloads/{ticker_symbol}_prices.csv')

In [1]:
# Spot price change vs Option price change - Scatter plot
import pandas as pd

# import plotly.express as px
import os

# 假设所有文件都存储在同一个目录下
directory = "./downloads/tsla_eod_2016"
files = [
    f for f in os.listdir(directory) if f.startswith("tsla_eod_") and f.endswith(".txt")
]

df = pd.DataFrame()
dtype_dict = {
    "[QUOTE_UNIXTIME]": "int64",
    "[QUOTE_READTIME]": "str",
    "[QUOTE_DATE]": "str",
    "[QUOTE_TIME_HOURS]": "float64",
    "[UNDERLYING_LAST]": "float64",
    "[EXPIRE_DATE]": "str",
    "[EXPIRE_UNIX]": "int64",
    "[DTE]": "float64",
    "[C_DELTA]": "float64",
    "[C_GAMMA]": "float64",
    "[C_VEGA]": "float64",
    "[C_THETA]": "float64",
    "[C_RHO]": "float64",
    "[C_IV]": "float64",  # 注意这里假设IV是float类型，原始数据中此列为空，可能需要预处理或设为object类型
    "[C_VOLUME]": "float64",
    "[C_LAST]": "float64",
    "[C_SIZE]": "str",
    "[C_BID]": "float64",
    "[C_ASK]": "float64",
    "[STRIKE]": "float64",
    "[P_BID]": "float64",
    "[P_ASK]": "float64",
    "[P_SIZE]": "str",
    "[P_LAST]": "float64",
    "[P_DELTA]": "float64",
    "[P_GAMMA]": "float64",
    "[P_VEGA]": "float64",
    "[P_THETA]": "float64",
    "[P_RHO]": "float64",
    "[P_IV]": "float64",  # 同C_IV
    "[P_VOLUME]": "float64",
    "[STRIKE_DISTANCE]": "float64",
    "[STRIKE_DISTANCE_PCT]": "float64",
}

# 循环读取每个文件
for file in files:
    file_path = os.path.join(directory, file)
    # 根据你的文件格式调整read_csv中的参数，如分隔符等
    data = pd.read_csv(file_path, dtype=dtype_dict, skipinitialspace=True)
    # 将读取的数据添加到all_data中
    df = pd.concat([df, data], ignore_index=True)

# 在这里，你需要根据实际的列名调整代码
# 计算股票和期权的日变化率等

# 示例：绘图（确保你已经计算了STOCK_DAY_CHANGE和OPTION_DAY_CHANGE列）
# fig = px.scatter(all_data, x='STOCK_DAY_CHANGE', y='OPTION_DAY_CHANGE', color='DTE', title='TSLA Stock vs Option Day Change (%)')
# fig.show()

df = df.sort_values(by="[QUOTE_UNIXTIME]").reset_index(drop=True)
# print(df[:5].to_string())
df["quote_date"] = pd.to_datetime(df["[QUOTE_DATE]"])

# df.set_index('quote_date', inplace=True)
# df['c_last_change'] = df['[C_LAST]'].pct_change()

In [ ]:
print(pd.__version__)

df.columns
df.index

In [2]:
import numpy as np
import pandas as pd

df["c_last"] = df["[C_LAST]"].where(df["[C_LAST]"] != 0, np.nan)

c_last_change = pd.Series(index=df.index, dtype=float)
underlying_last_change = pd.Series(index=df.index, dtype=float)

# 依據 '[EXPIRE_DATE]' 和 '[STRIKE]' 進行分組
groups = df.groupby(["[EXPIRE_DATE]", "[STRIKE]"])

for i, ((expire_date, strike), group) in enumerate(groups):
    changes = group["c_last"].pct_change(fill_method=None)
    c_last_change[changes.index] = changes

    changes = group["[UNDERLYING_LAST]"].pct_change(fill_method=None)
    underlying_last_change[changes.index] = changes

df["c_last_change"] = c_last_change
df["underlying_last_change"] = underlying_last_change

In [ ]:
# filtered = df[(df["[EXPIRE_DATE]"] == "2016-12-02") & (df["[STRIKE_DISTANCE]"] < 10)]
# groups = filtered.groupby(["[EXPIRE_DATE]", "[STRIKE]"])

# for (expire_date, strike), group in groups:
#     print(f"EXPIRE_DATE: {expire_date}, STRIKE: {strike}")
#     # print(group[['[STRIKE_DISTANCE]', '[STRIKE_DISTANCE_PCT]', '[UNDERLYING_LAST]', 'c_last', 'c_last_pct_change']])
#     print(
#         group[
#             [
#                 "[DTE]",
#                 "c_last",
#                 "c_last_change",
#                 "[UNDERLYING_LAST]",
#                 "underlying_last_change",
#             ]
#         ]
#     )

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

filtered = df[(df["[EXPIRE_DATE]"] == "2016-12-02") & (df["[STRIKE_DISTANCE]"] < 10)]

# 創建一個含有兩個子圖的圖表
fig = make_subplots(
    rows=1, cols=2, subplot_titles=("Days to Expire", "Strike Distance")
)

# 第一個子圖：使用到期日作為顏色維度
fig.add_trace(
    go.Scatter(
        x=filtered["underlying_last_change"],
        y=filtered["c_last_change"],
        mode="markers",
        marker=dict(
            size=15, color=filtered["[DTE]"], colorscale="Viridis", showscale=True
        ),
        text=filtered["[DTE]"],
    ),
    row=1,
    col=1,
)

# 第二個子圖：使用行權價差距作為顏色維度
fig.add_trace(
    go.Scatter(
        x=filtered["underlying_last_change"],
        y=filtered["c_last_change"],
        mode="markers",
        marker=dict(
            size=15,
            color=filtered["[STRIKE_DISTANCE]"],
            colorscale="Cividis",
            showscale=True,
        ),
        text=filtered["[STRIKE_DISTANCE]"],
    ),
    row=1,
    col=2,
)

# 更新布局設定
fig.update_layout(
    height=600, width=1200, title_text="Stock Price Change vs. Option Price Change"
)
fig.show()

In [38]:
import plotly.graph_objects as go

# 篩選出特定條件的資料
# filtered = df[(df['[EXPIRE_DATE]'] == '2016-12-02') & (df['[STRIKE_DISTANCE]'] < 10)]
filtered = df[(df["[EXPIRE_DATE]"] == "2016-12-02")]

# 初始化一個 plotly 圖形物件
fig = go.Figure()

# 對每一個 strike 畫線
for strike, group in filtered.groupby("[STRIKE]"):
    if 170 <= strike <= 200:
        fig.add_trace(
            go.Scatter(
                x=group["quote_date"],
                y=group["c_last"],
                mode="lines",
                name=f"Strike {strike}",
            )
        )

# 加上 UNDERLYING_LAST vs quote_date 的線，使用第二個 y 軸
fig.add_trace(
    go.Scatter(
        x=filtered["quote_date"],
        y=filtered["[UNDERLYING_LAST]"],
        mode="lines",
        name="Underlying Last",
        line=dict(color="black", dash="dash"),
        yaxis="y2",
    )
)

# 更新圖表的佈局來新增第二個 y 軸
fig.update_layout(
    title="c_last vs quote_date with Different Strikes and Underlying Last",
    xaxis_title="Quote Date",
    yaxis=dict(
        title="c_last",
    ),
    yaxis2=dict(title="Underlying Last", overlaying="y", side="right"),
    legend_title="Legend",
    height=800,
)

# 顯示圖表
fig.show()

In [ ]:
tsla = pd.read_csv("./downloads/TSLA_prices.csv")
tsla["spot_change"] = tsla["Adj Close"].pct_change()
tsla["date"] = pd.to_datetime(tsla["Date"])
tsla.set_index("date", inplace=True)
tsla

In [ ]:
import numpy as np

# df['[C_LAST]']
# df['[C_LAST]'].pct_change()
# df[:5].to_csv()

# df_slim = df[['quote_date', '[C_LAST]']]
merged_data = pd.merge(tsla, df, left_on="date", right_on="quote_date", how="inner")

# print(merged_data[:5].to_csv())
# df_slim
# df[['[C_LAST]']]
filtered = merged_data[merged_data["Date"] == "2016-01-05"]
# merged_data[['Date', 'quote_date']]
filtered = filtered[["Date", "spot_change", "c_last_change", "[STRIKE]", "[C_LAST]"]]
filtered = filtered[
    ~pd.isna(filtered["c_last_change"]) & ~np.isinf(filtered["c_last_change"])
]
filtered.sort_values("c_last_change", ascending=False)

# print(filtered[:5].to_csv())

In [ ]:
import plotly.express as px

fig = px.scatter(
    filtered,
    x="[STRIKE]",
    y="c_last_change",
    title="Tesla Spot Price Change vs Option Price Change",
)
fig.show()